In [ ]:
import theano
import theano.tensor as T
import theano.tensor.nnet as NN
import numpy as NP
import numpy.random as RNG
import h5py
import dataUtils
from collections import OrderedDict
from Wrapper import Model, rmsprop

In [ ]:
''' 
load data 
'''
in_dim = 49
out_dim = 49
batch_size = 16

#X_train, Y_train=read_txt('ptb.train.txt')
X_test, Y_test=dataUtils.read_txt('ptb.test.txt')


In [ ]:
def _step(trash, cur_in, rec_in1, rec_in2, rec_in3):
    gru1_h = model.gru(cur_in = cur_in, rec_in = rec_in1, name='gru1', shape=(in_dim, 20))
    gru2_h = model.gru(cur_in = gru1_h, rec_in = rec_in2, name='gru2', shape=(20, 20))
    gru3_h = model.gru(cur_in = gru2_h, rec_in = rec_in3, name='gru3', shape=(20, 20))
    merge = T.concatenate([gru1_h, gru2_h, gru3_h], axis=1)
    fc1 = model.fc(cur_in = merge, name='fc1', shape=(60, out_dim))
    return fc1, gru1_h, gru2_h, gru3_h
    

In [ ]:
model = Model()
char_in = T.tensor3()
char_tar = T.tensor3()
_char_in = char_in.dimshuffle(1, 0, 2)
starts = T.matrix()

sc, _ = theano.scan(_step, sequences=[_char_in], outputs_info=[starts, T.zeros((batch_size, 20)), T.zeros((batch_size, 20)), T.zeros((batch_size, 20))])
char_out = sc[0].dimshuffle(1, 0, 2)

cost = T.mean(T.sum((char_tar - char_out) ** 2, axis=2))

train_func = theano.function([char_in, starts, char_tar], [cost, char_out], updates=rmsprop(cost, model.weightsPack.getW_list()), allow_input_downcast=True)
#predic_func = theano.function([char_in, starts], char_out, allow_input_downcast=True)


In [ ]:
#jacobi = T.jacobian(char_out.flatten(), starts)
#jacobi_func = theano.function([char_in, starts], jacobi, allow_input_downcast=True)

In [ ]:
cost, char_out = train_func(X_train[0:16], X_train[0:16, 0], Y_train[0:16])